1. Take the input train data

2. Devide them into mini batches

3. Iterate over the mini_batchs
    
    a. Take a mini batch 
    
    b. iterate over the mini batch 
    
    c. run the feed forward algorithm 
    
    d. calculate the delta_bias and delta_weight 
       i.e. calculate the change in cost function w.r.t. change in bias and weight respectively
    
    e. don't back propagate for each input, just return the delta_bias, delta_w
    
    f. after finishing the mini batch iteration, tweak the biases and weights using the following formulas:
    
    ratio = eta / size_of_mini_batch
    
    w -> w' = w -  ratio * sum_of_all_delta_w
    
    b -> b' = b - ratio * sum_of_all_delta_b   
    
    g. After tweaking the changes dont run the feedforward again
    
    
    
    
    
    
    run the back propagation one time but over what ?
    
    he he he ! over the each of the input of the mini_batch 
    
    again do the same process
    
    
    Question :  
    When to stop  ?
    
    Ans :  1. stop when all the mini_batch outputs are correct 
           2. stop when the average cost error is less may be less then 0.2 or something else
           
           
           
    
    
    
    
    
    
    

In [114]:
import numpy as np
from keras.datasets import mnist
import random

from scipy import misc 
import imageio
import matplotlib.pyplot as plt

In [286]:
def sigmoid(x):
    
    return (1 / (1 + np.exp(-x)))

In [287]:
def sigmoid_derivative(x):
    result = (x - (x**2))
    #print("\n Sigmoid derivative is : ", result)
    return result

In [288]:
# this function split the array by choosing the size random indexes
#size of the new array
# return two arrays 
def splitArray(arr, size):
    
    #systemRandom = random.SystemRandom()
    #_index = np.array ( [ systemRandom.randint(0, len(arr)-1) for index in range(size) ] )

    _index = np.array ( [ index for index in random.sample(range(0,len(arr)), size)] )
    
    # choose the index values from the array
    new_arr = arr.take(_index)
    arr = np.delete(arr, _index)
    return arr, new_arr

In [351]:
def showImage(img):
    plt.imshow(img, "gray")

In [306]:
class Network:
    
    def __init__(self, layers_size, learning_rate, output_size):
        
        self.output_size = output_size
        self.layers_size = layers_size
        self.total_layers = len(layers_size)
        self.activation_layer = [] 
        self.eta = learning_rate
        self.thershold = 0.95
        
        # remember total number of rows are equal to the number of neurons in the next layer 
        self.weights = [  np.random.randn(row,col) for row, col in zip(self.layers_size[1:], self.layers_size[:-1]) ] 
        
        # 1-d vector
        self.biases =  [ np.random.randn(row, 1) for row in self.layers_size[1:] ]
        
    
    # x -> input layer
    def feed_forward(self, x): 
        
        self.activation_layer = [x]
        for w,b in zip(self.weights, self.biases):
            
            # w x  + b
            next_layer = sigmoid (np.dot(w, x) + b) 
            
            # storing the layerwise activation for using in the cost error calculations
            self.activation_layer.append(next_layer)
            
            x = next_layer 
            
#------------------------------------------------------------------------------------------------------------------------            
            
        
    # ( activation_layer[L] - Y(x) ) *  (sigmoid(z) - (sigmoid(z))^2)
    # ( Al - Y(x) ) * ( sigmoid(z) - (sigmoid(z)^2))
    def get_output_layer_error(self, expected_output):

        # since the layers variable contains the activation neuron for each layer 
        return (self.activation_layer[-1] - expected_output) * sigmoid_derivative(self.activation_layer[-1])

    
    # get the layer wise delta_l means layer wise error 
    # (transpose(weight[l+1])*delta_l(l+1)) * ( sigmoid(z) - (sigmoid(z)^2))
    def get_layerwise_error(self, expected_output):
    
        delta_l = [self.get_output_layer_error(expected_output)]

        
        #print("\nShape of output layer : ", np.shape(delta_l[0]))
        # suppose there are 3 layers suppose there are n layers 
        # according to our code      according to our code 
        # 0 to 1 are the weights   0 to n-2 are the weights        
        # so starting from 3-2 = 1   so starting from n-2 = 1 
        # delta_l contains layer + 1 error at starting index -> 0 
        
        for layer in range(self.total_layers-2, 0 ,-1):
            
            err = np.dot(np.transpose(self.weights[layer]), delta_l[0] )* sigmoid_derivative(self.activation_layer[layer])
            
            #print("\n err shape : ", np.shape(err))
            # insert error at the beginning
            delta_l.insert(0, err)
            
        return delta_l

            
    # this function is for getting the error
    def back_propagation(self, data, expected_output):
    
        delta_l = self.get_layerwise_error(expected_output)
    
        # calculate the change in weight and bias 
        
        # change in cost if there is a change in the bias
        delta_bias = delta_l
    
        delta_weight = [ np.zeros(np.shape(w)) for w in self.weights]
        
        # we have the delta_layers calculated, we have activation_layers then 
        # we can start from 0 to calculate the weight error
        # it is not a backward paas but we can take the advantage of already calculated values 
        
        delta_weight = [delta_weight[l]+np.dot(delta_l[l], np.transpose(self.activation_layer[l])) for l in range(self.total_layers-1)  ] 
    
        #print("\n Delta Weight Shape : ", np.shape(delta_weight[0]))
        # find the alternative way to push in front 
    
        #print("\n before return shape delta_weight:  ", np.shape(delta_weight))
        return delta_bias, delta_weight
    

    # this function tweak the weights and biases of the network 
    def update_weight_bias(self, mini_batches):


        #print("\n Weights Initialy : \n", self.weights)
        #print("\n Biases Initialy : \n", self.biases)
        
        
        
        #m = 1 
        # iterate over the mini batch 
        for mini_batch in mini_batches :
            
            avg_delta_bias = [ np.zeros(np.shape(b)) for b in self.biases ]
            avg_delta_weight = [ np.zeros(np.shape(w)) for w in self.weights ]
         
            
            # c = 0
            mini_batch_size = len(mini_batch)
            #print("\n mini batch size : ", mini_batch_size)
            # iterate over the inputs of a mini_batch 
            for data, label in mini_batch :

                self.feed_forward(data)
                
                #print("\n Feed forward : \n", self.activation_layer[-1])
            
                
                # get the change in bias and weight 
                # get the error 
                # either send the input as one hot vector or send the label as integer
                expected_output = np.zeros(shape=(self.output_size,1)) # shape chaning for testing 
                expected_output[label] = 1 
                #print("\n Expected output : ", expected_output)
                #print("\n Actual output : ", self.activation_layer[-1])
                
                delta_bias, delta_weight = self.back_propagation(data, expected_output)

                #print("\n Delta bias : \n", delta_bias)
                #print("\n Delta Weight : \n", delta_weight)
                
                
                #print("\navg : ",np.shape(avg_delta_weight))
                
                #print("\nnrml : ",np.shape(delta_weight))
                
                # remember the error biases and weights are also in the same dimension as avg one
                avg_delta_bias = np.add(avg_delta_bias, delta_bias)
                
                
                #print("\n avg_delta_weight : ",avg_delta_weight)
                #print("\n delta_weight : ",type(delta_weight))
                
                avg_delta_weight = [ np.add(avg_del_w, del_w) for avg_del_w, del_w in zip(avg_delta_weight, delta_weight) ]
                
                #print("\n Avg Delta bias : \n", delta_bias)
                #print("\n Avg Delta Weight : \n", delta_weight)
            
            #m += 1
            # now remove the average error from the weight and bias
            self.weights = [ weight - (self.eta/mini_batch_size) * avg_weight for weight, avg_weight in zip(self.weights, avg_delta_weight)] 
            self.biases = [ bias - (self.eta/mini_batch_size) * avg_bias for bias, avg_bias in zip(self.biases, avg_delta_bias)] 
    
            #print("\n Weights after \n", self.weights)
            #print("\n Biases after \n", self.biases)
            
            #print("\nlength of weights : ", len(self.weights))

    # this function is for training the data 
    def train_network(self, training_data, testing_data, mini_batch_size, ephoc) :


        for i in range(ephoc):
            
            print("\n Running for ephoc ", i)
            random.shuffle(training_data)

            # get the mini batches in the network 
            mini_batches = [ training_data[k:k+mini_batch_size] for k in range(0, len(training_data) , mini_batch_size)  ]

            self.update_weight_bias(mini_batches)
            self.test_network(testing_data)
        
        #print("Activation layer at last is : ", self.activation_layer[self.total_layers-1])
        
        

    # this is for the testing data
    def test_network(self, testing_data):

        count = 0
        for data, label in testing_data:

            self.feed_forward(data)

            output_layer = self.activation_layer[self.total_layers-1]
            
            v = [ np.where(output_layer > self.thershold) ] 

            #print("Value : ", v)
            if len(v[0][0]) == 1 and v[0][0] == label:
                count += 1
        
        print ("Ans is {0} / {1}".format(count ,len(testing_data)))
        

In [307]:
# send the data either in tuple form or in the x_train, y_train both should be in the vector

In [308]:
# read the input
(x_train, y_train),(x_test, y_test) = mnist.load_data()

shape = np.shape(x_train) 

#reshape the trained data into single vector 
new_x_train = np.reshape(x_train, newshape=( len(x_train), shape[1]*shape[2], 1) )

print(np.shape(new_x_train))

total_testing_data = len(x_test)  # 10000
total_training_data = len(x_train) - total_testing_data # 60000 - 10000

# call the feed forward to get the output layer 
#output_layer = feed_forward(hdr_nn, input_layer, 0)

(60000, 784, 1)


In [309]:
# normalize the data to make each value between 0 and 1
new_x_train = new_x_train/255

# split the data 
# 50000 and 10000

new_x_test = new_x_train[total_training_data:]
print(len(new_x_test))

new_y_test = y_train[total_training_data:]
print(len(new_y_test))

new_x_train = new_x_train[:total_training_data]
print(len(new_x_train))

new_y_train = y_train[:total_training_data]
print(len(new_y_train))


10000
10000
50000
50000


In [310]:
# converting the numpy array to list of tuples

training_data = [ (data, label) for data, label in zip(new_x_train, new_y_train)]
len(training_data)

testing_data = [ (data, label) for data, label in zip(new_x_test, new_y_test)]


In [395]:
not_trng_data = training_data[:50000]

not_testing_data = testing_data[:10000]

#number of neurons in layer
layer_wise_neuron_count = [784, 30, 10]

# create object for neural network
hdr_nn = Network(layer_wise_neuron_count, 1.0, 10)

hdr_nn.train_network(not_trng_data, not_testing_data, mini_batch_size=10, ephoc=20)


 Running for ephoc  0
Ans is 3826 / 10000

 Running for ephoc  1
Ans is 4832 / 10000

 Running for ephoc  2
Ans is 6087 / 10000

 Running for ephoc  3
Ans is 6412 / 10000

 Running for ephoc  4
Ans is 6801 / 10000

 Running for ephoc  5
Ans is 6848 / 10000

 Running for ephoc  6
Ans is 7252 / 10000

 Running for ephoc  7
Ans is 7363 / 10000

 Running for ephoc  8
Ans is 7547 / 10000

 Running for ephoc  9
Ans is 7539 / 10000

 Running for ephoc  10
Ans is 7681 / 10000

 Running for ephoc  11
Ans is 7491 / 10000

 Running for ephoc  12
Ans is 7701 / 10000

 Running for ephoc  13
Ans is 7787 / 10000

 Running for ephoc  14
Ans is 7885 / 10000

 Running for ephoc  15
Ans is 7882 / 10000

 Running for ephoc  16
Ans is 7919 / 10000

 Running for ephoc  17
Ans is 7973 / 10000

 Running for ephoc  18
Ans is 8070 / 10000

 Running for ephoc  19
Ans is 8148 / 10000


In [394]:
shape = np.shape(x_test)

#reshape the trained data into single vector 
real_testing_data = np.reshape(x_test, newshape=( len(x_test), shape[1]*shape[2], 1) )

real_testing_data = real_testing_data /255

real_testing_data = [(data, label) for data, label in zip(real_testing_data, y_test)]

hdr_nn.test_network(real_testing_data)

Ans is 8831 / 10000


In [396]:
hdr_nn.train_network(not_trng_data, not_testing_data, mini_batch_size=10, ephoc=20)


 Running for ephoc  0
Ans is 8154 / 10000

 Running for ephoc  1
Ans is 8129 / 10000

 Running for ephoc  2
Ans is 8143 / 10000

 Running for ephoc  3
Ans is 8256 / 10000

 Running for ephoc  4
Ans is 8288 / 10000

 Running for ephoc  5
Ans is 8205 / 10000

 Running for ephoc  6
Ans is 8290 / 10000

 Running for ephoc  7
Ans is 8205 / 10000

 Running for ephoc  8
Ans is 8246 / 10000

 Running for ephoc  9
Ans is 8285 / 10000

 Running for ephoc  10
Ans is 8321 / 10000

 Running for ephoc  11
Ans is 8407 / 10000

 Running for ephoc  12
Ans is 8398 / 10000

 Running for ephoc  13
Ans is 8346 / 10000

 Running for ephoc  14
Ans is 8347 / 10000

 Running for ephoc  15
Ans is 8362 / 10000

 Running for ephoc  16
Ans is 8451 / 10000

 Running for ephoc  17
Ans is 8466 / 10000

 Running for ephoc  18
Ans is 8449 / 10000

 Running for ephoc  19
Ans is 8495 / 10000


In [298]:
# testing the input against the small input

# 0 or 1 is the output value
train = [ (np.array([[0.4], [0.5], [0.6]]), 1), (np.array([[0.3], [0.8], [0.5]]), 0) ] 

#number of neurons in layer
layer_wise_neuron_count = [3, 2, 2]

# create object for neural network
hdr_nn = Network(layer_wise_neuron_count, 0.1)

hdr_nn.train_network(train, train, mini_batch_size=1, ephoc=3)




 Running for ephoc  0

 layer :  1

 layer :  1
Ans is 0 / 2

 Running for ephoc  1

 layer :  1

 layer :  1
Ans is 0 / 2

 Running for ephoc  2

 layer :  1

 layer :  1
Ans is 0 / 2
